In [1]:
import networkx as nx
import sys
import random
sys.path.append('../src/')
from random_graph import RandGraph
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [2]:
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, Oval, MultiLine, Circle
from bokeh.models import LinearColorMapper, ColumnDataSource
from bokeh.palettes import Spectral8, Spectral4
from bokeh.models.graphs import from_networkx
from bokeh.io import show, output_notebook

In [3]:

output_notebook()


Loading BokehJS ...

In [4]:
g = RandGraph(graph_type='simple')

In [89]:
plot = figure(title="Simple graph",
             tools="",
              x_range=(-1,1.5),
              y_range=(-.4,.4),
             toolbar_location=None)

In [5]:
graph = from_networkx(g.graph, nx.kamada_kawai_layout,
                     scale=2)

In [7]:
p = figure()

In [8]:
p.graph(graph)

TypeError: graph() missing 2 required positional arguments: 'edge_source' and 'layout_provider'

In [91]:
df = pd.DataFrame.from_dict({k:v for k,v in g.graph.nodes(data=True)},
                           orient='index')


In [92]:
df.capacity = 10 + df.capacity

In [93]:
df

,actors,capacity
1,None,NaN
2,None,NaN
3,None,17.0
4,None,16.0
5,None,12.0
6,None,12.0
7,None,NaN


In [124]:
l = []
for k,v in g.graph.nodes(data=True):
    if v['actors']:
        l.append(len(v['actors']))
    else:
        l.append(0)
l

[0, 0, 0, 0, 0, 0, 0]

In [125]:
df['color'] = l

In [134]:
df

,actors,capacity,color
1,None,NaN,0
2,None,NaN,0
3,None,17.0,0
4,None,16.0,0
5,None,12.0,0
6,None,12.0,0
7,None,NaN,0


In [126]:
source = ColumnDataSource(df)

In [127]:
graph.node_renderer.data_source = source

In [128]:
# Map cubehelix_palette
palette = sns.cubehelix_palette(10)
pal_hex_lst = palette.as_hex()

In [129]:
mapper = LinearColorMapper(palette=pal_hex_lst, low=0, high=21)

In [135]:
graph.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", 
                                      line_width=3)
graph.node_renderer.glyph = Circle(size='capacity',
                                   fill_color={'field':'color',
                                              'transform': mapper})

In [136]:
plot.renderers.append(graph)

In [137]:
plot.xgrid.visible =False
plot.ygrid.visible = False
plot.axis.visible = False

In [138]:
show(plot)

ERROR:bokeh.core.validation.check:E-1010 (CDSVIEW_SOURCE_DOESNT_MATCH): CDSView used by Glyph renderer must have a source that matches the Glyph renderer's data source: GlyphRenderer(id='24b31b0a-f000-4162-990c-2f7cb20bbab9', ...)


In [172]:
list(g.graph.nodes())

[1, 2, 7, 3, 4, 5, 6]

In [169]:
[v for k,v in g.graph.nodes(data=True)]

[{'actors': None},
 {'actors': [b'0e7e1a26']},
 {'actors': None},
 {'capacity': 7, 'actors': None},
 {'capacity': 6, 'actors': [b'0e7e1a26']},
 {'capacity': 2, 'actors': None},
 {'capacity': 2, 'actors': None}]

In [176]:
list(df.capacity)

[nan, nan, 17.0, 16.0, 12.0, 12.0, nan]

In [140]:
for step in range(100):
    g.action()


    plot = figure(title="Simple graph",
                 tools="",
                  x_range=(-1,1.5),
                  y_range=(-.4,.4),
                 toolbar_location=None)
    graph = from_networkx(g.graph, nx.kamada_kawai_layout,
                         scale=2)

    # size and colors
    df = pd.DataFrame.from_dict({k:v for k,v in g.graph.nodes(data=True)},
                               orient='index')
    df.capacity = 10 + df.capacity
    l = []
    for k,v in g.graph.nodes(data=True):
        if v['actors']:
            l.append(len(v['actors']))
        else:
            l.append(0)
    df['color'] = l

    # data source
    source = ColumnDataSource(df)
    graph.node_renderer.data_source = source

    # plot
    graph.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", 
                                          line_width=3)
    graph.node_renderer.glyph = Circle(size='capacity',
                                       fill_color={'field':'color',
                                                  'transform': mapper})

    plot.renderers.append(graph)
    plot.xgrid.visible =False
    plot.ygrid.visible = False
    plot.axis.visible = False
    show(plot)

In [192]:
def update():
    g.action()
    # size and colors
    df = pd.DataFrame.from_dict({k:v for k,v in g.graph.nodes(data=True)},
                               orient='index')
    df.capacity = 10 + df.capacity
    l = []
    for k,v in g.graph.nodes(data=True):
        if v['actors']:
            l.append(len(v['actors']))
        else:
            l.append(0)
    df['color'] = l
    new = {'capacity': list(df.capacity),
          'color': list(df.color),
          'actors': list(df.actors),
          'index': list(df.index)}
    source.stream(new)

In [199]:
def make_doc(doc):
    g = RandGraph(graph_type='simple')
    plot = figure(title="Simple graph",
             tools="",
              x_range=(-1,1.5),
              y_range=(-.4,.4),
             toolbar_location=None)
    graph = from_networkx(g.graph, nx.kamada_kawai_layout,
                         scale=2)
    # data source
    source = ColumnDataSource({'capacity': [],
          'color': [],
          'actors': []})
    graph.node_renderer.data_source = source

    def update():
        g.action()
        # size and colors
        df = pd.DataFrame.from_dict({k:v for k,v in g.graph.nodes(data=True)},
                                   orient='index')
        df.capacity = 10 + df.capacity
        l = []
        for k,v in g.graph.nodes(data=True):
            if v['actors']:
                l.append(len(v['actors']))
            else:
                l.append(0)
        df['color'] = l
        new = {'capacity': list(df.capacity),
              'color': list(df.color),
              'actors': list(df.actors),
              'index': list(df.index)}
        source.stream(new)
    
    # plot
    graph.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", 
                                          line_width=3)
    graph.node_renderer.glyph = Circle(size='capacity',
                                       fill_color={'field':'color',
                                                  'transform': mapper})

    plot.renderers.append(graph)
    plot.xgrid.visible =False
    plot.ygrid.visible = False
    plot.axis.visible = False
    
    doc.title = 'hello world!'
    doc.add_periodic_callback(update, 100)
    doc.add_root(plot)

In [200]:
apps = {'/': Application(FunctionHandler(make_doc))}

In [201]:
server.stop()

In [202]:

server = Server(apps, port=5000)
server.start()

In [203]:
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models.widgets import TextInput, Button, Paragraph

# create some widgets
button = Button(label="Say HI")
input = TextInput(value="Bokeh")
output = Paragraph()

# add a callback to a widget
def update():
    output.text = "Hello, " + input.value
button.on_click(update)

# create a layout for everything
layout = column(button, input, output)

# add the layout to curdoc
curdoc().add_root(layout)